# Get EMDB experimental annotations 

In [29]:
import os
import requests
import logging as logger
from bs4 import BeautifulSoup
from subprocess import Popen

**NOTE** 

This doesn't work with EMDB! It produces no output.

```python
url = 'https://www.ebi.ac.uk/emdb/api/annotations/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
```    

In [31]:
myurl='https://www.ebi.ac.uk/emdb/api/entry/experiment/EMD-2984'
resp = requests.get(url=myurl)
data = resp.json()

In [43]:
data['structure_determination_list']['structure_determination'][0].keys()

dict_keys(['structure_determination_id', 'method', 'aggregation_state', 'specimen_preparation_list', 'microscopy_list', 'image_processing', 'instance_type'])

**NOTES**

To gather all the data I need to compile a list of the currently available non-obsolete entries in EMDB
* Option 1: Take a for loop running serially up to a high number. IDs take a 4-digit numeric format only and seem to run serially, except for a few which I suspect are obsolete. 
* Option 2: Compile a list from here https://files.rcsb.org/pub/emdb/structures/ . Try `ftplib`?

Ruben's functions. Taking bits and pieces, but mostly ideas.

In [ ]:
@retry_decorator
def retrieveInfoFromEMDB(emdbId):
    resp = requests.get(url=EMDB_INFO_QUERY % emdbId)
    if resp.status_code != 200:
        logger.error("request status %d" % (resp.status_code))
        raise DownloadException("Error, retrieveInfoFromEMDB request status %d" % (resp.status_code))
    else:
        data = resp.json()[emdbId][0]
        try:
            pdbs = data["deposition"]["fitted_pdb_id_list"]
            pdbs = [elem.lower() for elem in pdbs["pdb_id"]]
        except KeyError:
            try:
                pdbs = [elem["pdb_id_list"]["pdb_id"][0].lower() for elem in data["experiment"]["fitting"]]
            except KeyError:
                logger.error("%s has not associated pdb" % (emdbId))
                raise DownloadException("Error, %s has not associated pdb" % (emdbId))

        voxel_size = data["map"]["pixel_spacing"]["y"]["value"]
        max_val_vox = data["map"]["statistics"]["maximum"]
        grid_shape = [int(elem) for elem in data["map"]["dimensions"].values()]

        axis_order = data["map"]["axis_order"]
        axis_order = [axis_order["slow"], axis_order["medium"], axis_order["fast"]]
        coords_origin = data["map"]["origin"]
        coords_origin = [coords_origin["section"], coords_origin["row"],
                         coords_origin["column"]]  # order slow, medium, fast
        try:
            resolution = float(data["processing"]["reconstruction"]["resolution_by_author"])
        except KeyError:
            raise DownloadException("Error, no resolution available for %s " % (emdbId))

        date = data["deposition"]["deposition_date"]
        info = {"voxel_size": voxel_size, "max_val_vox": max_val_vox, "grid_shape": grid_shape,
                "coords_origin": coords_origin,
                "resolution": resolution, "pdbs": pdbs, "axis_order": axis_order, "date": date}
        if resolution > config.WORSE_RESOLUTION_ALLOWED:
            msg = "Error downloading %s. Too bad resolution:\n" % (emdbId,)
            logger.error(msg)
            raise DownloadException(msg)
            # writeErrorFile(emdbId, "Error downloading %s. Too bad resolution:\n" % (emdbId,), info, pdbId="")

        if len(set(grid_shape)) > 1:
            logger.error(f"Error downloading {emdbId}: Just cubic volumes are considered")
            raise DownloadException("Error downloading %s. Just cubic volumes are considered:\n" % (emdbId,))
        return info

In [ ]:
@retry_decorator
def downloadEMDB(emdbId, mapsOutDir):
    outName = os.path.join(mapsOutDir, emdbId + '.map')
    logger.info("Trying to download emdb %s to %s" % (emdbId, outName))
    if not os.path.isfile(outName):
        emd, num = emdbId.split("-")
        cmd = ('wget -qO- ' + EMDB_DONWLOAD_QUERY + ' --timeout=' +str(TIMEOUT_TIME)+
               '  | zcat  > %s') % (emdbId, num, outName + ".tmp")
        logger.info(cmd)
        p = Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True, executable="/bin/bash")
        out = p.communicate()
        if len(out[1]) > 0:
            logger.error(out)
            tryToRemove(outName + ".tmp")
            msg = "Error downloading (wget) emdbId %s to %s. %s" % (emdbId, outName, out)
            raise DownloadException(msg)
        os.rename(outName + ".tmp", outName)
    else:
        logger.warn("Already downloaded emdbId %s ->%s " % (emdbId, outName))

    return outName

@retry_decorator
def downloadEMDB_halfMaps(emdbId, mapsOutDir, onlyAverage=False):
    outName = os.path.join(mapsOutDir, emdbId + '.map')
    logger.info("Trying to download emdbID %s to %s" % (emdbId, outName))
    halfFnames = []
    if not os.path.isfile(outName):
        try:
            time.sleep(random.randint(0,TIMEOUT_TIME))
            ftp = ftplib.FTP(EMDB_FTP_SERVER)
            ftp.login()
            try:
                ftp.cwd(EMDB_FTP_DIR % emdbId)
            except ftplib.all_errors as e:
                msg = f"FTTP error for {emdbId}. It probabaly does not contain half-maps: " + str(e)
                logger.error(msg)
                raise DownloadException(msg)
            fnames = ftp.nlst()
            half_names_inServer = [None, None]
            logger.info("Downaloading half maps: "+ " "+" ".join(fnames))
            for fname in fnames:
                match_objs = re.match(".*half[-_\.]*(map_?)*([12])", fname)
                if match_objs:
                    half_names_inServer[int(match_objs.group(2)) - 1] = fname
            logger.info(half_names_inServer)
            if None in half_names_inServer:
                raise DownloadException(f"Error, half map not available for {emdbId}")
            tmp_halfs = []
            tmp_half_name = None
            for halfNum in range(1, 3):
                try:
                    tmp_half_name = outName + "_half_%d.tmp.mrc" % halfNum
                    tmp_halfs.append(tmp_half_name)
                    with  BytesIO() as flo:
                        ftp.retrbinary('RETR ' + half_names_inServer[halfNum - 1], flo.write)
                        flo.seek(0)
                        with open(tmp_half_name, 'wb') as fout, gzip.GzipFile(fileobj=flo) as fgzip:
                            shutil.copyfileobj(fgzip, fout)
                except ftplib.all_errors as e:
                    logger.error(f"FTTP error for {emdbId}: "+str(e))
                    raise e
                    # if tmp_half_name:
                    #     tryToRemove(tmp_half_name)
                    # msg = "Error downloading emdbId %s to %s: %s" % (emdbId, outName, str(e))
                    # raise DownloadException(msg)
        except ftplib.all_errors as e:
            logger.error(f"FTTP error for {emdbId}: " + str(e))
            raise e
            # msg = "Error downloading emdbId %s to %s: %s" % (emdbId, outName, str(e))
            # raise DownloadException(msg)

        data = loadVol(tmp_halfs[0])[0]
        data += loadVol(tmp_halfs[1])[0]
        data /= 2.
        saveVol(outName, data, fname_headerLike=tmp_halfs[0])
        if onlyAverage:
            for tmp_half_name in tmp_halfs:
                tryToRemove(tmp_half_name)
        else:
            for tmp_half_name in tmp_halfs:
                new_fname = tmp_half_name.replace(".tmp", "")
                os.rename(tmp_half_name, new_fname)
                halfFnames.append(new_fname)
    else:
        logger.warn("Already downloaded emdbId %s ->%s " % (emdbId, outName))

    return outName, halfFnames